# DSCI 572 Lab 1

A meta-comment: in the first half of this course (and thus this lab), we break away from the typical MDS approach of using/interpreting data science software and start digging a bit into the implementation of such software. If you don't like this, please keep in mind that (1) it's just one lab, (2) a higher-than-usual fraction of the content has been made optional, and (3) I find it hard to imagine that the concepts here will turn out to be completely useless. If you like this, enjoy!  

In lab 2 we'll return more to the style of DSCI 571 and other courses. 

In [1]:
import numpy as np
import scipy.linalg as spla
import scipy.optimize as spo
import sklearn.datasets

### Exercise 1a: gradients of mathematical functions
rubric={reasoning:5}

Compute the gradient of each of the following mathematical functions. The notation for the gradient of a function $f$ is $\nabla f(x)$. Note that $x$ may be a vector but $f$ returns a scalar in all the cases below. The gradient is also a vector.

In some cases, the dimension of $x$ is provided: for example, "$x \in \mathbb{R}^3$" means that $x$ is a vector with 3 elements. In other cases, you should be able to infer the dimension from the context (for example, for $f_2$ we can infer that $x \in \mathbb{R}^2$ since otherwise the matrix multiplication wouldn't make sense). Finally, in some cases (like $f_6$) the dimension is unknown but you should be able to give an answer that holds regardless of the dimension of $x$. 

Hint: for $\nabla f_5(x)$ you can write $x^\top A x$ as a sum of a few terms. For $\nabla f_6(x)$ it may be helpful to write out $x^\top x$ using summation notation, take the derivative, and then put it back into vector notation at the end.

1. $f_1(x) = \sin(x)$ where $x\in \mathbb{R}$
2. $f_2(x) = [0\;\; 1]x$
3. $f_3(x) = \exp(x_1 + x_2x_3)$ where $x \in \mathbb{R}^3$
4. $f_4(x) = \exp(x_1 + x_2x_3)$ where $x \in \mathbb{R}^4$
5. $f_5(x) = x^\top A x$ where $A=\left[ \begin{array}{cc}1 & 2 \\0 & -3 \end{array} \right]$
6. $f_6(x) = x^\top x$
7. $f_7(x) = x_1^2\sin(x_2)$ where $x \in \mathbb{R}^2$

### Exercise 1b: gradients of Python functions
rubric={code:2}

Write a Python function that computes the gradient of each of the following Python functions. You can use [scipy gradient checker](http://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.optimize.check_grad.html) to check your results for a few values of the inputs, as shown in the example below. 

Note: all of the functions we deal with here return a scalar, regardless of the size of the input vector `x`. We will not consider the case where the output itself is a vector, since it is not relevant for our context of loss function minimization. 

A note about terminology: the word "dimension" is a disaster, truly. When we have our math hats on, we say a vector of length 5 is 5-dimensional, it lives in $\mathbb{R}^5$, etc. But when we have our code hats on, all vectors are 1-dimensional numpy arrays (regardless of their length); 2-dimension means a matrix. I don't know what to say... it's a failure of mankind. If you're confused, please ask. 

Having said the above: you do not need to consider the case where the input is a 2 (or higher) dimensional **numpy array**; just treat the input is just a vector (1-d numpy array). But the code should work regardless of the length ("dimension") of the input vector.  

Finally, note that only the first two are required; the remaining are optional.

In [ ]:
# EXAMPLE
def example(x):
    return np.sum(x**2)

def example_grad(x):
    return 2.0*x

x0 = np.random.rand(5)
diff = spo.check_grad(example, example_grad, np.array(x0))
if diff < 1e-5:
    print('Success (probably)')
else:
    print('Gradient incorrect (probably)')

In [ ]:
def foo(x):
    return np.sum(x)

def foo_grad(x):
    pass

In [ ]:
def pin(x):
    return np.sin(x[1])

def pin_grad(x):
    pass

In [ ]:
w = np.random.rand(100)

def zap(x):
    return w@x # this is matrix multiplication; equivalent to %*% in R.

def zap_grad(x):
    pass

In [ ]:
################
### OPTIONAL ###
################

def baz(x):
    result = 0
    for i in range(len(x)):
        result = result + x[i]**i
    return result

def baz_grad(x):
    pass

In [ ]:
################
### OPTIONAL ###
################

def bar(x):
    if np.abs(x[1]) > 2:
        return 0
    else:
        return -(x[0]*x[0]+1)*np.cos(x[1]-1)

def bar_grad(x):
    pass

### Exercise 2: gradient descent
rubric={code:3}

Write a function `gradient_descent` that takes in a function `f`, its gradient `f_grad`, and a starting point `x0`, and returns a local minimum of `f`. Also write some documentation above the function.

The code below tests your function by comparing the results to [a builtin scipy minimizer](http://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize).

In [ ]:
# DOCUMENTATION GOES HERE
def gradient_descent(f, f_grad, x0, learning_rate=0.001, opt_tol=0.01, max_iter=10000):
    x = x0
    
    n = 0
    while True:
        n += 1
        
        # CODE GOES HERE (hint: it's not a lot of code!)
        #
        
        if spla.norm(f_grad(x)) < opt_tol or n >= max_iter:
            return x

# testing, using functions from Exercise 1b
print(gradient_descent(example, example_grad, np.zeros(2)))
print(spo.minimize(example, np.zeros(2), jac=example_grad).x)

### Exercise 3: logistic regression 
rubric={code:3,reasoning:2}

The code below implements part of a logistic regression classifier. Making use of your `gradient_descent` function above, complete the code and use it to perform logistic regression on the digit data set from Lab 1. We will start with binary (2-class) classification, so train your classifier to predict whether or not a digit is a 2.

Report your training and validation error. Also, show some examples of where the method makes mistakes on the validation set. Does it trip up on numbers that look more like 2's?

Notice that the code isn't written in object-oriented style. It's just a bit cleaner this way, because of the way we interact with the `gradient_descent` function.

Notice also that there's no "intercept" (what you call $\beta_0$ in regression) here. This is just for simplicity. But it would not be difficult to add. 

In [ ]:
def predict_lr(W, X):
    return np.sign(X@W)

def loss_lr(W, X, y):
    yXw = y * (X@W)
    return np.sum(np.log(1 + np.exp(-yXw)))

def loss_lr_grad(W, X, y):
    yXw = y * (X@W)
    return -X.T @ (y/(1+np.exp(yXw)))

# DOCUMENTATION GOES HERE
def fit_lr(X, y):
    # YOUR CODE HERE (hint: it's not a lot of code!)
    return np.random.randn(X.shape[1]) # this is a placeholder just returning random junk; delete it.

# load up the data set
digits = sklearn.datasets.load_digits()
X = digits['data'][:1000]
y = digits['target'][:1000]
y_2_vs_all = np.ones(y.size)
y_2_vs_all[y != 2] = -1                 # 2's are encoded as +1 and non-2's are encoded as -1
Xvalidate = digits['data'][1000:]
yvalidate = digits['target'][1000:]
yvalidate_2_vs_all = np.ones(yvalidate.size)
yvalidate_2_vs_all[yvalidate != 2] = -1 # see above

# check gradient of loss
spo.check_grad(lambda W: loss_lr(W,X,y_2_vs_all), lambda W: loss_lr_grad(W,X,y_2_vs_all), np.random.randn(X.shape[1]))

# test the code
W = fit_lr(X, y_2_vs_all)
pred = predict_lr(W, Xvalidate)
valid_acc = np.sum(pred == yvalidate_2_vs_all)/len(pred)
print('Validation accuracy: %.1f%%' % (valid_acc*100))


### (optional; more effort than usual, but good) Exercise 3b: multi-class
rubric={code:1,reasoning:1}

Make a copy of your code from Exercise 3, but now improve it so that it handles multi-class (>2 classes) classification. Try your multi-class code out on the full digit data set and report/discuss your results. 

How it works: the way this works is that each class gets its own $w$ vector. So we now have a matrix $W$ whose size is $D\times C$ (or $C\times D$, up to you), where $D$ is the input dimensionality and $C$ is the number of classes. (You'll notice that, weirdly, the binary classification case is _not_ the special case where $C=2$, because with binary classification we can get away with just one vector of weights.) To get the class probabilities, you'll need to use the [softmax function](https://en.wikipedia.org/wiki/Softmax_function), which basically amounts to taking $\exp$ of each output and then dividing by the sum so that everything adds up to 1 and can be interpreted as probabilities. Hard predictions are made by taking the maximum of these values. The loss function is then the negative log of the _true class_ probability (summed over training examples). 

Note: you'll also need to worry about numerical issues here. You can read about the problem and solution [here](https://hips.seas.harvard.edu/blog/2013/01/09/computing-log-sum-exp/) and you can make use of [scipy.misc.logsumexp](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.misc.logsumexp.html) if you wish. 


### Exercise 4: scikit-learn logistic regression
rubric={reasoning:2}

The scikit-learn implementation of logistic regression can be found [here](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Compare your implementation to the sklearn one, both in terms of speed and accuracy, on the 2 vs. non-2 binary classification problem. For a fair comparison, set `C=1e6`. If you feel like making a more fair comparison, you can set `fit_intercept=False` since your code above doesn't fit an intercept. Or you can take this opportunity to try both (with and without intercept) and see if the intercept makes a any difference in this case (but that's optional).


### Exercise 4b: multi-class with sklearn
rubric={reasoning:2}

Here's an interesting question: using sklearn's logistic regression classifier (setting `C=1e6` or not, it shouldn't matter), solve the multi-class problem, so that your predictions go from 0 to 9. Given these predictions, how well do you do _on the 2 vs. non-2 problem_ as compared to the binary classifier, in terms of training error? Give an intuitive explanation of the results. I would actually recommend trying to think about what you expect the result to be (before trying it), but it's not a requirement. Think about [lecture 1](https://github.ubc.ca/ubc-mds-2016/DSCI_572_sup-learn-2_students/blob/master/lectures/lecture1.ipynb).

Oh, by the way, when you try it you'll see that the training error is zero for both cases, which isn't very illuminating. The problem is "easy" and we can (over)fit the data perfectly. To make it harder, let's use just a subset of the features, for example `X[:,:10]` instead of `X`. (So, yes, regarding Mat F's question about $n\ll d$ in DSCI 571's Slack channel: features are good!)

### Exercise 5: don't forget that logistic regression is a GLM
rubric={writing:2,reasoning:2}

In DSCI 562 you studied generalized linear models (GLMs), a class that includes logistic regression. Write one paragraph about your experience studying/using logistic regression from the statistics angle vs. the machine learning angle. Draw an interesting connection, give an example of something that confused you, or comment on how the viewpoints differ. Are machine learning and statistics the same thing? Why/why not?


<br><br>

----------------

**FYI: This is the line between the first half of the lab (lecture 1 and 2) and the second half (lectures 3 and 4).** 

However, in terms of time expenditure, I'd say the first half of this lab is more work than the second half.

----------------

<br><br>


## Exercise 6: roundoff errors
rubric={reasoning:7}

For each of the code snippets below, explain the result. The first three are answered for you, so you can get a sense of what types of explanations we're looking for. Note that in both the second and third case we get the "expected" result (zero), but the explanations are very different.

In [ ]:
0.3 - 0.2 - 0.1

(Above) The result is not zero because 0.3, 0.2, and 0.1 are not represented exactly as floating point numbers.

In [ ]:
0.5 - 0.25 - 0.125 - 0.125

(Above) The result is zero because 0.5, 0.25, and 0.125 are powers of 2 and there they _are_ represented exactly as floating point numbers. There is no roundoff error present. 

In [ ]:
0.4 - 0.2 - 0.2

(Above) The result is correct (zero) despite the fact that 0.4 and 0.2 are not represented exactly as floating point numbers. This is a case of good luck: while 0.4 and 0.2 are not represented exactly, the roundoff errors happened to cancel out during the subtractions.

#### snippet (a)
<!-- rubric={reasoning:1} -->

In [ ]:
30-20-10

#### snippet (b)
<!-- rubric={reasoning:1} -->

In [ ]:
64-32-16-16

#### snippet (c)
<!-- rubric={reasoning:1} -->

In [ ]:
(10.0**100 + 1) - 10.0**100

#### snippet (d)
<!-- rubric={reasoning:1} -->

In [ ]:
(10.0**100000 + 1) - 10.0**100000

#### snippet (e)
<!-- rubric={reasoning:1} -->

In [ ]:
(np.exp(1000) + 1) - np.exp(1000)

#### snippet (f)
<!-- rubric={reasoning:1} -->

In [2]:
sum(np.zeros(10)+0.1)

0.99999999999999989

#### snippet (g)
<!-- rubric={reasoning:1} -->

In [ ]:
np.sin(np.pi)

#### snippet (h)
<!-- rubric={reasoning:1} -->

In [ ]:
x = np.ones(100000)
x[0] = 1e20

y = np.ones(100000)
y[-1] = 1e20

sum(x) == sum(y)

Hint for snippet (h) above: see the "order of operations" section of the [lecture notes](https://github.ubc.ca/ubc-mds-2016/DSCI_572_sup-learn-2_students/blob/master/lectures/lecture3.ipynb)!!!!

#### (optional) snippet (i)
<!-- rubric={reasoning:1} -->

In [ ]:
f = lambda x: np.sqrt(1+x**2)
g = lambda x: x * np.sqrt(1+1/x**2)

x = 10.0**np.arange(0,501,10)
f(x) == g(x)

#### (optional) snippet (j)

In [ ]:
import numpy as np
x = np.zeros(10)+0.1
sum(x) == np.sum(x)

Hint for snippet (j) above: see [Pairwise summation](https://en.wikipedia.org/wiki/Pairwise_summation). 

### (optional) Exercise 7: logistic regression and log-sum-exp
rubric={reasoning:1,code:1}

In multi-class logistic regression, your loss ends up having one term for each class, so you get something of the form $\log \sum_{n=1}^N \exp(x_n)$. We can rewrite this as 

$\log \displaystyle\sum_{n=1}^N \exp(x_n) = a+ \log \displaystyle\sum_{n=1}^N \exp(x_n-a)$

for any $a$. Make sure you understand why this is the case before proceeding.

1. Explain why this formulation might be more numerically stable and why $\max_n x_n$ is a sensible choice for $a$.
1. This trick seems to rely on the fact that overflow is more of a danger than underflow, because we may now compute $\exp(z)$ for $z\ll -1$. Explain why overflow is more of a problem than underflow here.
2. In the previous exercise we used the approximation $\log(1+\exp(x))\approx x$. Is that just a specific case of what we have here? It seems that what we did earlier was an approximation, whereas what we did here is mathematically exact. Do you think there is any significance to this distinction, in practice?
3. Write a python function `log_sum_exp` that takes in an array `x` and computes the above expression. Then, (if you did Exercise 3b) plug it into your multi-class implementation from 3b. Does it help?

## Exercise 8: finite differences
rubric={reasoning:2}

The `scipy.optimize.minimize` function takes in the derivative information through the optional argument `jac` (for Jacobean). As shown below, the code works even if you don't pass in the derivative. 


In [ ]:
print(minimize(example, np.zeros(2)).x)

1. Explain how optimization is possible even without having access to the gradient of the function. 
2. Give one advantage and one disadvantage of providing the gradient to an optimization routine.

<br><br>

----------------

**For the final Exercise, you can choose which one you want to do, version A or version B. If you do both, you will get modest Spark points.**

----------------

<br><br>


## Exercise 9: numerical differentiation _OR_ automatic differentiation
rubric={reasoning:1,code:1}


#### Exercise 9A: numerical differentiation

1. Earlier, you used the [scipy gradient checker](https://docs.scipy.org/doc/scipy-0.17.0/reference/generated/scipy.optimize.check_grad.html). Implement your own gradient checker, that has the same interface as the scipy one (i.e. takes arguments `func`, `grad`, `x0` and returns the norm of the difference.) So long as you **cite your sources**, you may refer to, or even take code from, the [lecture 4 notes](https://github.ubc.ca/ubc-mds-2016/DSCI_572_sup-learn-2_students/blob/master/lectures/lecture4.ipynb) and/or the [scipy gradient checker source code](https://github.com/scipy/scipy/blob/v0.17.0/scipy/optimize/optimize.py#L625-L671) and/or the [scipy finite differences source code](https://github.com/scipy/scipy/blob/v0.18.1/scipy/optimize/optimize.py#L633-L688).
2. Under what circumstances might the gradient checker not work as intended, or give misleading results? Give two counterexamples: one in which the gradient is correct but your gradient checker thinks the gradient is wrong, and one in which the gradient is wrong but your gradient checker test passes.


#### Exercise 9B: AutoGrad


1. Install [AutoGrad](https://github.com/HIPS/autograd) using `pip install autograd`. Then, reproduce the `tanh` example on the front page, but change the colours of the plot to show that you actually ran the code. Your submission should include the plot.
2. Earlier, you fit parameters for logistic regression using a gradient function that was derived by hand. Here, copy over the model code but not the gradient code, and use AutoGrad to handle the derivatives. Compare your results to those of Exercise 3. Comment on accuracy, speed, code quality, etc.

Note for the second part: 
The `loss_lr` function has the potential for overflow as discussed in Lecture 3. This doesn't end up mattering in Exercise 3 because we don't need `loss_lr`, we only need `loss_lr_grad`, which is numerically "safe" in comparison. However, in 9B Autograd actually uses `loss_lr` itself, and chokes on the Inf/NaNs. To get around this, you can do the following:

In [1]:
from autograd.scipy.misc import logsumexp

# Takes a 1-d numpy array x and returns an array of the same size with the operation
# np.log(1+np.exp(x_i) applied to each element x_i, done in a numerically safe way. 
def log_one_plus_exp_safe(x):
        return logsumexp(np.concatenate((np.zeros((1,x.size)),x[None])),axis=0)
    
# logistic regression loss using "safe" log(1+exp(x)) function
def loss_lr(W, X, y):
    yXw = y * (X@W)
    return np.sum(log_one_plus_exp_safe(-yXw))
